#### - Sobhan Moradian Daghigh
#### 
#### - 1/20/2022
#### 
#### - PR - EX03 - Q3 - All Parts

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = pd.read_csv('./inputs/P3/BOM.csv')
dataset.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

### Preprocessing

In [4]:
dataset = dataset.dropna(subset = ["RainTomorrow"])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

### Determine 'month' attr

In [5]:
month = []
for _, row in dataset.iterrows():
    month.append(int(row.Date.split('-')[1]))

dataset['month'] = month
dataset.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,month
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,12
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,12
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,12
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,12
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,12


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

### Fill Nans

In [7]:
dataset = dataset.apply(lambda x: x.fillna(x.mean()) if x.dtype == np.float64 else x.fillna(x.mode()[0]), axis=0)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        142193 non-null  float64
 3   MaxTemp        142193 non-null  float64
 4   Rainfall       142193 non-null  float64
 5   Evaporation    142193 non-null  float64
 6   Sunshine       142193 non-null  float64
 7   WindGustDir    142193 non-null  object 
 8   WindGustSpeed  142193 non-null  float64
 9   WindDir9am     142193 non-null  object 
 10  WindDir3pm     142193 non-null  object 
 11  WindSpeed9am   142193 non-null  float64
 12  WindSpeed3pm   142193 non-null  float64
 13  Humidity9am    142193 non-null  float64
 14  Humidity3pm    142193 non-null  float64
 15  Pressure9am    142193 non-null  float64
 16  Pressure3pm    142193 non-null  float64
 17  Cloud9am       142193 non-nul

### Part A.

In [8]:
# Using https://www.projectpro.io/recipes/drop-out-highly-correlated-features-in-python
cor_matrix = dataset.corr().abs()
cor_matrix

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,month
MinTemp,1.000000,0.733919,0.103314,0.353698,0.051907,0.173331,0.174946,0.174187,0.232372,0.005913,0.424357,0.434034,0.061460,0.015902,0.897999,0.699828,0.203826
MaxTemp,0.733919,1.000000,0.074202,0.442214,0.331505,0.066329,0.014586,0.050381,0.499777,0.499725,0.309093,0.397422,0.226776,0.214469,0.880087,0.969735,0.164435
Rainfall,0.103314,0.074202,1.000000,0.038186,0.173011,0.127250,0.085977,0.056762,0.221392,0.249609,0.159676,0.120366,0.171613,0.145848,0.011384,0.077553,0.030710
Evaporation,0.353698,0.442214,0.038186,1.000000,0.290375,0.147371,0.140669,0.094134,0.377594,0.289915,0.211823,0.231341,0.152224,0.147763,0.417323,0.429195,0.021086
Sunshine,0.051907,0.331505,0.173011,0.290375,1.000000,0.022479,0.005685,0.039471,0.351980,0.446849,0.030932,0.015477,0.537714,0.559379,0.210174,0.349271,0.017322
WindGustSpeed,0.173331,0.066329,0.127250,0.147371,0.022479,1.000000,0.577864,0.658377,0.209272,0.025822,0.426600,0.384642,0.052281,0.080202,0.146756,0.032274,0.058868
WindSpeed9am,0.174946,0.014586,0.085977,0.140669,0.005685,0.577864,1.000000,0.513079,0.269027,0.030991,0.215178,0.165050,0.019171,0.041080,0.128769,0.005026,0.050776
WindSpeed3pm,0.174187,0.050381,0.056762,0.094134,0.039471,0.658377,0.513079,1.000000,0.144270,0.015789,0.277495,0.239743,0.041369,0.019803,0.162144,0.028443,0.060405
Humidity9am,0.232372,0.499777,0.221392,0.377594,0.351980,0.209272,0.269027,0.144270,1.000000,0.659852,0.131601,0.176168,0.354394,0.274613,0.471137,0.492430,0.090079
Humidity3pm,0.005913,0.499725,0.249609,0.289915,0.446849,0.025822,0.030991,0.015789,0.659852,1.000000,0.025769,0.048566,0.400006,0.407898,0.217593,0.555777,0.018315


In [9]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(bool))
upper_tri

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,month
MinTemp,NaN,0.733919,0.103314,0.353698,0.051907,0.173331,0.174946,0.174187,0.232372,0.005913,0.424357,0.434034,0.061460,0.015902,0.897999,0.699828,0.203826
MaxTemp,NaN,NaN,0.074202,0.442214,0.331505,0.066329,0.014586,0.050381,0.499777,0.499725,0.309093,0.397422,0.226776,0.214469,0.880087,0.969735,0.164435
Rainfall,NaN,NaN,NaN,0.038186,0.173011,0.127250,0.085977,0.056762,0.221392,0.249609,0.159676,0.120366,0.171613,0.145848,0.011384,0.077553,0.030710
Evaporation,NaN,NaN,NaN,NaN,0.290375,0.147371,0.140669,0.094134,0.377594,0.289915,0.211823,0.231341,0.152224,0.147763,0.417323,0.429195,0.021086
Sunshine,NaN,NaN,NaN,NaN,NaN,0.022479,0.005685,0.039471,0.351980,0.446849,0.030932,0.015477,0.537714,0.559379,0.210174,0.349271,0.017322
WindGustSpeed,NaN,NaN,NaN,NaN,NaN,NaN,0.577864,0.658377,0.209272,0.025822,0.426600,0.384642,0.052281,0.080202,0.146756,0.032274,0.058868
WindSpeed9am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513079,0.269027,0.030991,0.215178,0.165050,0.019171,0.041080,0.128769,0.005026,0.050776
WindSpeed3pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144270,0.015789,0.277495,0.239743,0.041369,0.019803,0.162144,0.028443,0.060405
Humidity9am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.659852,0.131601,0.176168,0.354394,0.274613,0.471137,0.492430,0.090079
Humidity3pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025769,0.048566,0.400006,0.407898,0.217593,0.555777,0.018315


In [10]:
high_cor = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
high_cor

['Pressure3pm', 'Temp3pm']

In [11]:
dataset = dataset.drop(high_cor, axis=1)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        142193 non-null  float64
 3   MaxTemp        142193 non-null  float64
 4   Rainfall       142193 non-null  float64
 5   Evaporation    142193 non-null  float64
 6   Sunshine       142193 non-null  float64
 7   WindGustDir    142193 non-null  object 
 8   WindGustSpeed  142193 non-null  float64
 9   WindDir9am     142193 non-null  object 
 10  WindDir3pm     142193 non-null  object 
 11  WindSpeed9am   142193 non-null  float64
 12  WindSpeed3pm   142193 non-null  float64
 13  Humidity9am    142193 non-null  float64
 14  Humidity3pm    142193 non-null  float64
 15  Pressure9am    142193 non-null  float64
 16  Cloud9am       142193 non-null  float64
 17  Cloud3pm       142193 non-nul

### Label Encoding

In [12]:
[print('\n\nColumn: {} - count: {}\n'.format(col, len(dataset[col].unique())), 
       dataset[col].unique().tolist()[:20]) for col in dataset.columns if dataset[col].dtype == np.object_]



Column: Date - count: 3436
 ['2008-12-01', '2008-12-02', '2008-12-03', '2008-12-04', '2008-12-05', '2008-12-06', '2008-12-07', '2008-12-08', '2008-12-09', '2008-12-10', '2008-12-11', '2008-12-12', '2008-12-13', '2008-12-14', '2008-12-16', '2008-12-17', '2008-12-18', '2008-12-19', '2008-12-20', '2008-12-21']


Column: Location - count: 49
 ['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree', 'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond', 'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown', 'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat', 'Bendigo']


Column: WindGustDir - count: 16
 ['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', 'ENE', 'SSE', 'S', 'NW', 'SE', 'ESE', 'E', 'SSW']


Column: WindDir9am - count: 16
 ['W', 'NNW', 'SE', 'ENE', 'SW', 'SSE', 'S', 'NE', 'N', 'SSW', 'WSW', 'ESE', 'E', 'NW', 'WNW', 'NNE']


Column: WindDir3pm - count: 16
 ['WNW', 'WSW', 'E', 'NW', 'W', 'SSE', 'ESE', 'ENE', 'NNW', 'SSW', 'SW', 'SE', 'N', 'S', '

[None, None, None, None, None, None, None]

### Ok, first of all, Im gonna drop the 'data' attr
### Then, Im gonna use a OHE for 'RainToday'
### And finally a LabelEncoding for others 

In [13]:
dataset = dataset.drop('Date', axis=1)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   Evaporation    142193 non-null  float64
 5   Sunshine       142193 non-null  float64
 6   WindGustDir    142193 non-null  object 
 7   WindGustSpeed  142193 non-null  float64
 8   WindDir9am     142193 non-null  object 
 9   WindDir3pm     142193 non-null  object 
 10  WindSpeed9am   142193 non-null  float64
 11  WindSpeed3pm   142193 non-null  float64
 12  Humidity9am    142193 non-null  float64
 13  Humidity3pm    142193 non-null  float64
 14  Pressure9am    142193 non-null  float64
 15  Cloud9am       142193 non-null  float64
 16  Cloud3pm       142193 non-null  float64
 17  Temp9am        142193 non-nul

In [14]:
def OHEer(dataset, col):
    pd.get_dummies(dataset.loc[:, col])
    attr = pd.get_dummies(dataset[col], prefix= col + '_')
    dataset = pd.merge(
        left=dataset,
        right=attr,
        left_index=True,
        right_index=True,)
    dataset.pop(col)
    return dataset

In [15]:
dataset = OHEer(dataset, 'RainToday')
dataset.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,Temp9am,RainTomorrow,month,RainToday__No,RainToday__Yes
0,Albury,13.4,22.9,0.6,5.469824,7.624853,W,44.0,W,WNW,...,71.0,22.0,1007.7,8.000000,4.503167,16.9,No,12,1,0
1,Albury,7.4,25.1,0.0,5.469824,7.624853,WNW,44.0,NNW,WSW,...,44.0,25.0,1010.6,4.437189,4.503167,17.2,No,12,1,0
2,Albury,12.9,25.7,0.0,5.469824,7.624853,WSW,46.0,W,WSW,...,38.0,30.0,1007.6,4.437189,2.000000,21.0,No,12,1,0
3,Albury,9.2,28.0,0.0,5.469824,7.624853,NE,24.0,SE,E,...,45.0,16.0,1017.6,4.437189,4.503167,18.1,No,12,1,0
4,Albury,17.5,32.3,1.0,5.469824,7.624853,W,41.0,ENE,NW,...,82.0,33.0,1010.8,7.000000,8.000000,17.8,No,12,1,0


In [16]:
label_encoder = LabelEncoder()
dataset.Location = label_encoder.fit_transform(dataset.Location)
dataset.WindGustDir = label_encoder.fit_transform(dataset.WindGustDir)
dataset.WindDir3pm = label_encoder.fit_transform(dataset.WindDir3pm)
dataset.WindDir9am = label_encoder.fit_transform(dataset.WindDir9am)
dataset.RainTomorrow = label_encoder.fit_transform(dataset.RainTomorrow)
dataset['RainTomorrow'] = dataset.pop('RainTomorrow')
dataset.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,Temp9am,month,RainToday__No,RainToday__Yes,RainTomorrow
0,2,13.4,22.9,0.6,5.469824,7.624853,13,44.0,13,14,...,71.0,22.0,1007.7,8.000000,4.503167,16.9,12,1,0,0
1,2,7.4,25.1,0.0,5.469824,7.624853,14,44.0,6,15,...,44.0,25.0,1010.6,4.437189,4.503167,17.2,12,1,0,0
2,2,12.9,25.7,0.0,5.469824,7.624853,15,46.0,13,15,...,38.0,30.0,1007.6,4.437189,2.000000,21.0,12,1,0,0
3,2,9.2,28.0,0.0,5.469824,7.624853,4,24.0,9,0,...,45.0,16.0,1017.6,4.437189,4.503167,18.1,12,1,0,0
4,2,17.5,32.3,1.0,5.469824,7.624853,13,41.0,1,7,...,82.0,33.0,1010.8,7.000000,8.000000,17.8,12,1,0,0


### Splite Dataset into train, test, validation

In [36]:
X, x_test, y, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.1, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

### Part B.

In [22]:
ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(x_train, y_train)

sfs_forward = SequentialFeatureSelector(ridge, n_features_to_select=10, direction="forward").fit(x_train, y_train)
sfs_backward = SequentialFeatureSelector(ridge, n_features_to_select=10, direction="backward").fit(x_train, y_train)

In [23]:
X.columns[sfs_forward.get_support()]

Index(['MaxTemp', 'Rainfall', 'Sunshine', 'WindGustSpeed', 'WindDir9am',
       'WindSpeed3pm', 'Humidity3pm', 'Pressure9am', 'Cloud3pm',
       'RainToday__Yes'],
      dtype='object')

In [24]:
X.columns[sfs_backward.get_support()]

Index(['MinTemp', 'MaxTemp', 'Sunshine', 'WindGustSpeed', 'WindDir9am',
       'WindSpeed3pm', 'Humidity3pm', 'Pressure9am', 'Cloud3pm',
       'RainToday__Yes'],
      dtype='object')

### Part C

#### Im gonna use forward sfs

In [25]:
features = X.columns[sfs_forward.get_support()]

In [37]:
new_x_train = pd.DataFrame([x_train.loc[:, c] for idx, c in enumerate(features)]).T
new_x_test = pd.DataFrame([x_test.loc[:, c] for idx, c in enumerate(features)]).T
new_x_val = pd.DataFrame([x_val.loc[:, c] for idx, c in enumerate(features)]).T

new_x_train

,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindDir9am,WindSpeed3pm,Humidity3pm,Pressure9am,Cloud3pm,RainToday__Yes
60708,36.4,0.0,7.624853,57.0,0.0,15.0,22.0,1014.9,4.503167,0.0
81500,15.6,0.0,5.300000,15.0,3.0,7.0,81.0,1021.9,4.503167,0.0
81526,11.8,0.8,1.000000,17.0,7.0,11.0,84.0,1033.3,4.503167,0.0
61413,16.9,0.0,1.800000,31.0,4.0,2.0,73.0,1017.2,6.000000,0.0
76656,18.4,0.0,9.900000,26.0,3.0,17.0,69.0,1021.9,4.503167,0.0
...,...,...,...,...,...,...,...,...,...,...
137857,27.0,8.2,0.000000,39.0,3.0,9.0,75.0,1008.1,8.000000,1.0
134506,24.7,0.0,7.624853,37.0,3.0,19.0,43.0,1012.6,4.503167,0.0
145452,20.7,0.0,7.624853,41.0,9.0,26.0,32.0,1028.1,7.000000,0.0
44815,18.0,0.2,7.624853,35.0,12.0,26.0,71.0,1036.1,4.503167,0.0


#### KNN

In [62]:
k = list(range(1, 101, 2))
param_grid = {'n_neighbors': k}

knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, cv=2, scoring='accuracy', verbose=1)
grid.fit(new_x_val, y_val)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


GridSearchCV(cv=2, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
                                         23, 25, 27, 29, 31, 33, 35, 37, 39, 41,
                                         43, 45, 47, 49, 51, 53, 55, 57, 59, ...]},
             scoring='accuracy', verbose=1)

In [82]:
best_k = grid.best_params_.get('n_neighbors')
best_k

31

In [83]:
knn = KNeighborsClassifier(n_neighbors= best_k)
knn.fit(new_x_train, y_train)
print("Accuracy of test dataset: {}".format(knn.score(new_x_test, y_test)))

Accuracy of test dataset: 0.8487341772151898


### Part D.

In [103]:
rasht = pd.DataFrame(data=[[28.6, 0.4, 9.88, 65, 6, 16, 73, 1010.855, 5.9, 0]],
                     columns=['MaxTemp', 'Rainfall', 'Sunshine', 'WindGustSpeed', 'WindDir9am',
                              'WindSpeed3pm', 'Humidity3pm', 'Pressure9am', 'Cloud3pm', 'RainToday__Yes'])
rasht

,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindDir9am,WindSpeed3pm,Humidity3pm,Pressure9am,Cloud3pm,RainToday__Yes
0,28.6,0.4,9.88,65,6,16,73,1010.855,5.9,0


In [104]:
print("Accuracy of test dataset: {}".format(knn.predict_proba(rasht)))

Accuracy of test dataset: [[0.19354839 0.80645161]]


### FINITO